# Second Feature Engineering

Downloading the new dataset of ibovespa and from our dataset load the stocks of years 2019 until 2023. We already implemented and its stored in our data folder.

In [1]:
## retrieving ibovespa data
## Do not need to run, we download it already

import yfinance as yf
import pandas as pd
import os
from pathlib import Path

ibov = "^BVSP"
df_ibov = yf.download(ibov, start="2019-01-01", end="2023-11-20")
df_ibov = df_ibov[['Close']].reset_index()
df_ibov.rename(columns={'Date': 'date', 'Close': 'ibovespa_close'}, inplace=True)
Path("../data").mkdir(exist_ok=True)
#df_ibov.to_csv("../data/ibovespa_2019-2023.csv", index=False)
#df_ibov

C:\Users\phili\AppData\Local\Temp\ipykernel_34308\3208783800.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_ibov = yf.download(ibov, start="2019-01-01", end="2023-11-20")
[*********************100%***********************]  1 of 1 completed


In [2]:
# load ibov
ibov_df = pd.read_csv("../data/ibovespa_2019-2023.csv")

In [3]:
## load stocks data

tickers = [
    'PETR4', 'VALE3', 'ITUB4', 'BBDC4', 'ABEV3', 'BBAS3', 'GGBR4', 'BRAP4', 'LREN3', 'MGLU3',
    'B3SA3', 'WEGE3', 'JBSS3', 'SUZB3', 'RADL3', 'ELET3', 'ELET6', 'SANB11', 'RENT3', 'RAIL3',
    'VIVT4', 'KLBN11', 'HYPE3', 'CSAN3', 'UGPA3', 'BRFS3', 'BRKM5', 'CIEL3', 'TOTS3', 'ENBR3'
]

base_dir = "../data/base"
all_dfs = []

for year in range(2019, 2024):
    file_path = os.path.join(base_dir, f"{year}_brazil_stocks.csv")
    if os.path.exists(file_path):
        print(f"Loading {file_path}...")
        df_year = pd.read_csv(file_path, low_memory=False)
        df_year['date'] = pd.to_datetime(df_year['date'], format='%Y%m%d', errors='coerce')
        all_dfs.append(df_year)
    else:
        print(f"Warning: {file_path} not found — skipping.")

if not all_dfs:
    raise FileNotFoundError("No yearly stock data files were found (2019–2023).")

stock_df = pd.concat(all_dfs, ignore_index=True)
stock_df = stock_df[stock_df['ticker'].isin(tickers)]

cols_to_drop = [
    'currency', 'name', 'marketType', 'bdiCode', 'prazoT', 'paperSpecification',
    'optionPrice', 'priceCorrection', 'paperDueDate', 'quoteFactor'
]
stock_df = stock_df.drop(columns=cols_to_drop, errors='ignore')
stock_df['date'] = pd.to_datetime(stock_df['date'])

Loading ../data/base\2019_brazil_stocks.csv...
Loading ../data/base\2020_brazil_stocks.csv...
Loading ../data/base\2021_brazil_stocks.csv...
Loading ../data/base\2022_brazil_stocks.csv...
Loading ../data/base\2023_brazil_stocks.csv...


In [4]:
## merge with Ibovespa
ibov_df = pd.read_csv("../data/ibovespa_2019-2023.csv")
ibov_df.rename(columns={'Date': 'date'}, inplace=True)
ibov_df['date'] = pd.to_datetime(ibov_df['date'])

merged = stock_df.merge(ibov_df[['date', 'ibovespa_close']], on='date', how='left')
df = merged.copy()

## Feature Selection

In [5]:
## feature Engineering

df['day_of_week'] = df['date'].dt.day_name()
day_map = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5}
df['day_of_week'] = df['day_of_week'].map(day_map)

df['daily_return'] = (df['close'] - df['open']) / df['open']
df['price_range'] = df['max'] - df['min']
df['volume_per_quantity'] = df['volume'] / df['quantity']

df['tomorrow'] = df.groupby('ticker')['close'].shift(-1)
df['target'] = (df['tomorrow'] > df['close']).astype(int)

# Rolling metrics
df['rolling_close_5']   = df.groupby('ticker')['close'].transform(lambda x: x.shift(1).rolling(5).mean())
df['rolling_std_5']     = df.groupby('ticker')['close'].transform(lambda x: x.shift(1).rolling(5).std())
df['rolling_return_5']  = df.groupby('ticker')['daily_return'].transform(lambda x: x.shift(1).rolling(5).mean())
df['rolling_volume_5']  = df.groupby('ticker')['volume'].transform(lambda x: x.shift(1).rolling(5).mean())
df['momentum_5']        = df['close'] / df['rolling_close_5'] - 1


In [6]:
horizons = [2, 5, 55, 220]
for h in horizons:
    df[f"Close_Ratio_{h}"] = df.groupby('ticker')['close'].transform(lambda x: x / x.rolling(h).mean())
    df[f"Trend_{h}"] = df.groupby('ticker')['target'].transform(lambda x: x.shift(1).rolling(h).sum())

df = df.dropna(subset=df.columns[df.columns != "tomorrow"])


# Evaluating Models with backtesting

In [7]:
# Train/test model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
features = [
    'quantity', 'volume', 'ibovespa_close', 'day_of_week',
    'price_range', 'volume_per_quantity',
    'rolling_std_5', 'rolling_return_5', 'momentum_5', 'rolling_volume_5',
    'Close_Ratio_5', 'Close_Ratio_55', 'Close_Ratio_220',
    'Trend_5', 'Trend_55'
]

train = df.iloc[:-500]
test = df.iloc[-500:]

model = RandomForestClassifier(n_estimators=100, random_state=1)
model.fit(train[features], train['target'])
preds = model.predict(test[features])
print("Precision:", precision_score(test['target'], preds))


Precision: 0.6379310344827587


The Random Forest model achieved a precision of 0.63, indicating that its “up” predictions were correct about 63% of the time.

However, since financial data is sequential and non-stationary, a single train-test split may not represent the model’s true performance over time. And such precision could be result of overfitting, which you'll soon understand as one of the major problems we've had during this project.

To address this, we adopted a walk-forward (backtesting) strategy, where we train on the first year and test on the next, then progressively expand the training set.
This approach ensures that:

- The model only uses past data to predict the future (avoiding data leakage).
- We can evaluate how well it generalizes to new market conditions over time.

In [8]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["target"], preds], axis=1)
    return combined

In [9]:
import time
from tqdm import tqdm  # progress bar

def backtest(data, model, predictors, start=50, step=1000):
    all_predictions = []

    # tqdm adds a progress bar in notebooks or terminal
    for i in tqdm(range(start, data.shape[0], step), desc="Backtesting Progress"):
        t0 = time.time()

        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)

        elapsed = time.time() - t0
        print(f"Iteration up to index {i:5d} | train size: {len(train):5d} | took {elapsed:.2f} sec")

    return pd.concat(all_predictions)


In [10]:
predictions = backtest(df, model, features)

Backtesting Progress:   3%|▎         | 1/29 [00:00<00:08,  3.16it/s]

Iteration up to index    50 | train size:    50 | took 0.31 sec


Backtesting Progress:   7%|▋         | 2/29 [00:01<00:14,  1.84it/s]

Iteration up to index  1050 | train size:  1050 | took 0.70 sec


Backtesting Progress:  10%|█         | 3/29 [00:02<00:22,  1.14it/s]

...

Iteration up to index 26050 | train size: 26050 | took 18.72 sec


Backtesting Progress:  97%|█████████▋| 28/29 [04:20<00:17, 17.91s/it]

Iteration up to index 27050 | train size: 27050 | took 20.45 sec


Backtesting Progress: 100%|██████████| 29/29 [04:42<00:00,  9.73s/it]

Iteration up to index 28050 | train size: 28050 | took 21.33 sec


In [11]:
predictions["Predictions"].value_counts()

Predictions
0    15071
1    13866
Name: count, dtype: int64

In [12]:
precision_score(predictions["target"], predictions["Predictions"])

0.5768786960911583

In [13]:
predictions["target"].value_counts() / predictions.shape[0]

target
0    0.507171
1    0.492829
Name: count, dtype: float64

## Backtesting (Walk-Forward Validation)

After introducing **backtesting (walk-forward validation)**, the model evaluation became more realistic and representative of how it would perform in real-world trading scenarios.

### Why Backtesting Matters

* **Chronological Training:**
  Each training period uses only past data to predict the next horizon, avoiding future data leakage.

* **Realistic Simulation:**
  This setup mirrors how the model would actually be deployed in live trading, being retrained periodically as new data becomes available.

### Comparing Results

Even though the overall precision dropped from approximately **0.61** to **0.57**, the newer result is more trustworthy because:

* The earlier score was based on a single, fixed test set, which could reflect a lucky period rather than consistent predictive power.
* The backtesting result reflects average real-world performance across multiple time periods, making it more robust and less overfitted.


In [14]:
horizons = [2, 5, 55, 220] ## two days, a week, a month, a year worth of trade
new_predictors = []

for horizon in horizons:
    # Create rolling ratio (close vs rolling mean of close)
    ratio_column = f"Close_Ratio_{horizon}"
    df[ratio_column] = df["close"] / df["close"].rolling(horizon).mean()

    # Create rolling trend (sum of past 'target' values)
    trend_column = f"Trend_{horizon}"
    df[trend_column] = df["target"].shift(1).rolling(horizon).sum()

    new_predictors += [ratio_column, trend_column]

# Combine your base features with the new ones


features = [
   'quantity', 'volume',
    'ibovespa_close', 'day_of_week', 'price_range', 'volume_per_quantity',
    'rolling_close_5', 'rolling_std_5', 'momentum_5', 'rolling_volume_5'
] + new_predictors

print("Final feature list:")
print(features)


Final feature list:
['quantity', 'volume', 'ibovespa_close', 'day_of_week', 'price_range', 'volume_per_quantity', 'rolling_close_5', 'rolling_std_5', 'momentum_5', 'rolling_volume_5', 'Close_Ratio_2', 'Trend_2', 'Close_Ratio_5', 'Trend_5', 'Close_Ratio_55', 'Trend_55', 'Close_Ratio_220', 'Trend_220']


### Added Features

Here we added two new features to improve the model's ability to capture both short-term and long-term market patterns:

- **Close_Ratio_h:**
  Measures the relative price position — how far above or below the recent average the price is.
  This helps identify potential overbought or oversold conditions.

- **Trend_h:**
  Captures momentum by counting how many times the price increased in the past *h* sessions.
  This helps the model detect sustained uptrends or downtrends.

In [15]:
df = df.dropna(subset=df.columns[df.columns != "tomorrow"])

In [16]:
df

,date,ticker,open,close,min,max,avg,quantity,volume,ibovespa_close,...,rolling_volume_5,momentum_5,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_55,Trend_55,Close_Ratio_220,Trend_220
6820,2019-11-28,ELET6,35.05,35.78,34.91,35.82,35.54,1103400,39217044.0,108290.0,...,75080881.4,-0.001507,1.016911,1.0,1.064121,4.0,0.912023,34.0,0.941900,122.0
6821,2019-11-28,ENBR3,19.29,19.56,19.24,19.60,19.48,3769000,73451232.0,108290.0,...,35752440.0,0.022692,0.706903,2.0,0.611326,4.0,0.498122,34.0,0.516735,122.0
6822,2019-11-28,GGBR4,17.39,17.17,17.02,17.58,17.31,8195700,141885362.0,108290.0,...,431928483.4,0.014536,0.934931,2.0,0.506759,4.0,0.444218,34.0,0.455172,123.0
6823,2019-11-28,HYPE3,33.52,33.58,33.40,33.82,33.55,888900,29825024.0,108290.0,...,64010091.6,-0.020249,1.323350,1.0,1.193489,3.0,0.872657,33.0,0.890148,123.0
6824,2019-11-28,ITUB4,34.45,34.60,34.02,34.70,34.42,15850200,545589332.0,108290.0,...,723271224.0,-0.015031,1.014960,0.0,1.229654,3.0,0.896872,32.0,0.916943,123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35612,2023-11-17,ITUB4,30.29,30.45,30.12,30.59,30.45,24575400,748326813.0,125062.0,...,882559210.0,0.026220,0.730303,0.0,0.956404,0.0,1.046163,15.0,1.084597,120.0
35613,2023-11-17,MGLU3,2.22,2.22,2.12,2.33,2.21,297371000,659930643.0,125062.0,...,503911766.0,0.207835,0.135904,0.0,0.085057,0.0,0.077172,14.0,0.079333,119.0
35614,2023-11-17,GGBR4,24.34,24.51,24.26,24.71,24.57,12991400,319205611.0,125062.0,...,329996779.2,0.034788,1.833895,0.0,0.918872,0.0,0.855492,13.0,0.877765,118.0
35615,2023-11-17,WEGE3,33.51,33.26,32.85,33.67,33.18,8938000,296624415.0,125062.0,...,266163011.2,0.006415,1.151463,0.0,1.159855,0.0,1.153172,12.0,1.192698,118.0


# Test model

In [17]:
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)

In [18]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >=.6] = 1
    preds[preds <.6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["target"], preds], axis=1)
    return combined

In [19]:
predictions = backtest(df, model, new_predictors)

Backtesting Progress:   3%|▎         | 1/29 [00:00<00:16,  1.66it/s]

Iteration up to index    50 | train size:    50 | took 0.60 sec


Backtesting Progress:   7%|▋         | 2/29 [00:01<00:22,  1.19it/s]

Iteration up to index  1050 | train size:  1050 | took 1.01 sec


Backtesting Progress:  10%|█         | 3/29 [00:03<00:31,  1.19s/it]

...

Iteration up to index 26050 | train size: 26050 | took 19.81 sec


Backtesting Progress:  97%|█████████▋| 28/29 [04:33<00:18, 18.80s/it]

Iteration up to index 27050 | train size: 27050 | took 20.47 sec


Backtesting Progress: 100%|██████████| 29/29 [04:54<00:00, 10.16s/it]

Iteration up to index 28050 | train size: 28050 | took 20.81 sec


In [20]:
predictions["Predictions"].value_counts()

Predictions
0.0    25516
1.0     3201
Name: count, dtype: int64

In [21]:
precision_score(predictions["target"], predictions["Predictions"])

0.6091846298031866

In [22]:
predictions["target"].value_counts() / predictions.shape[0]

target
0    0.507644
1    0.492356
Name: count, dtype: float64

### Backtesting with New Features

After introducing the new features (`Close_Ratio_h` and `Trend_h`) and applying the walk-forward (backtesting) approach, the model achieved a **precision of approximately 0.60**.

- **Precision:** 0.60
  When the model predicts that the price will rise, it is correct about 60% of the time.
  This is an improvement over earlier backtesting runs (~0.59 precision), showing that the new features helped the model better capture meaningful market patterns.
  It's important to mention that no evaluation tests were performed here, they'll be displayed in the EvaluationSecondModels.ipynb. Hence we should assume here that the model might (probably) be overfitting due to its high precision.

#### Why Performance Improved

The new features (`Close_Ratio_h` and `Trend_h`) capture both **relative price levels** and **momentum** across multiple time horizons.
This allows the Random Forest model to:

- Recognize **short-term corrections** and **long-term trends**.
- Respond to **multi-scale market dynamics**, improving its ability to generalize.
- Produce more **stable and realistic predictions** when evaluated through walk-forward validation.

Overall, combining richer temporal features with proper backtesting leads to **more reliable and actionable model performance** in a time-series trading context.



Delete features that are not needed anymore;

In [23]:
drop_features = ['open', 'close', 'min', 'max', 'avg', 'daily_return', 'rolling_close_5', 'Trend_220', 'Close_Ratio_2']

df.drop(drop_features, axis=1, inplace=True)